# Explicação da estratégia de previsão:

Para realizar a previsão da nota vou utilizar o conjunto de dados que tratei durante a análise exploratória, utilizando a coluna 'IMDB_Rating' como variável dependente e as colunas numéricas('Released_Year', 'Runtime', 'Meta_score', 'No_of_Votes', 'Gross') e as colunas categóricas(colunas Certificate e de Genre codificadas) como variáveis independentes. 

Como estamos querendo prever uma nota, ou seja, um valor numérico, estamos trabalhando com um problema de Regressão, para isso, vamos testar 4 modelos e comparar os valores das métricas R2 e RMSE(Root Mean Squared Error) para avaliá-los. Os modelos testados são: 

- Regressão Linear simples (Modelo Baseline)
- Regressão Polinomial 
- XGBoost
- RandomForestRegressor

Todos esses, exceto o modelo baseline, que serve de comparação, passam por um processo de GridSearch aplicado somente no conjunto de treino para seleção de hiperparâmetros, em seguida são avaliados no conjunto de testes com o conjunto de hiperparâmetros vencedor(pela métrica R2).

O modelo que apresentou o melhor resultado foi o XGBoost, que teve um R2 de aproximadamente 0,60, que foi superior a todos os outros modelos. O segundo melhor modelo apresentado foi o RandomForestRegressor, que chegou perto com um R2 de 0,57. 

In [31]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import joblib



In [2]:
filmes = pd.read_csv("dados_filmes_tratados.csv")
filmes.head()

,Series_Title,Released_Year,Runtime,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,...,Romance,Sci-Fi,Sport,Thriller,War,Western,Certificate_Livre,Certificate_Nao classificado,Certificate_Orientacao parental recomendada,Soma_notas
0,The Godfather,1972,175.0,9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,...,0,0,0,0,0,0,False,False,False,14907376.4
1,The Dark Knight,2008,152.0,9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,...,0,0,0,0,0,0,False,False,True,20729088.0
2,The Godfather: Part II,1974,202.0,9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,...,0,0,0,0,0,0,False,False,False,10169568.0
3,12 Angry Men,1957,96.0,9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,...,0,0,0,0,0,0,True,False,False,6208605.0
4,The Lord of the Rings: The Return of the King,2003,201.0,8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,...,0,0,0,0,0,0,True,False,False,14620546.2


# Modelos

#### Seleção de features e split holdout(Treino e teste):

In [3]:
features = []

for col in filmes.columns:
    if col not in ['Series_Title', 'Overview', 'Director', 'Star1', 'Star2', 'Star3', 'Star4','Soma_notas','IMDB_Rating', 'Imputed_Meta_score', 'Imputed_Gross']:
        features.append(col)

# Após alguns testes, os modelos que obtiveram melhor resultado não utilizaram as colunas imputadas de Metascore e Gross, 
# portanto apenas retirei as entradas nulas.

filmes = filmes[~((filmes['Gross'].isnull()) | (filmes['Meta_score'].isnull()))] 
filmes_X = filmes[features]
filmes_y = filmes['IMDB_Rating']

X_treino, X_teste, y_treino, y_teste = train_test_split(filmes_X, filmes_y, test_size=0.3, random_state=0)

#### Teste com modelo simples de comparação (baseline)

In [7]:
modelo_baseline = LinearRegression()
modelo_baseline.fit(X_treino, y_treino)

predicao_baseline = modelo_baseline.predict(X_teste)

rmse = np.sqrt(mean_squared_error(y_teste, predicao_baseline))
r2 = r2_score(y_teste, predicao_baseline)

In [8]:
print(f'RMSE = {rmse}')
print(f'R2 = {r2}')

RMSE = 0.19465296784291247
R2 = 0.5332851560645706


#### Teste com Gridsearch em modelo de regressão polinomial

In [10]:
modelo_polinomial = Pipeline([('normalizacao', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('linear', LinearRegression())
])

grid_parametros_polinomial = {'poly__degree': [1,2,3,4]}

grid_search_polinomial = GridSearchCV(modelo_polinomial, grid_parametros_polinomial, cv = 10, scoring='r2')
grid_search_polinomial.fit(X_treino,y_treino)

print(f'Melhor parâmetro para o grau: {grid_search_polinomial.best_params_}')

Melhor parâmetro para o grau: {'poly__degree': 1}


In [11]:
predicao_polinomial = grid_search_polinomial.best_estimator_.predict(X_teste)
rmse = np.sqrt(mean_squared_error(y_teste, predicao_polinomial))
r2 = r2_score(y_teste, predicao_polinomial)

print("Métricas da melhor Regressão Polinomial no conjunto de teste:\n")

print(f'RMSE = {rmse}')
print(f'R2 = {r2}')

Métricas da melhor Regressão Polinomial no conjunto de teste:

RMSE = 0.1946529678431366
R2 = 0.5332851560634959


#### Teste com Gridsearch no modelo XGBoost

In [12]:
modelo_xgboost = XGBRegressor(random_state=0)

grid_parametros_xgboost = {'n_estimators': [100, 200, 300], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]}

grid_search_xgboost= GridSearchCV(modelo_xgboost, grid_parametros_xgboost, cv = 10, scoring='r2', n_jobs=-1)

grid_search_xgboost.fit(X_treino,y_treino)

,estimator,"XGBRegressor(...ree=None, ...)"
,param_grid,"{'learning_rate': [0.01, 0.1, ...], 'max_depth': [3, 5, ...], 'n_estimators': [100, 200, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,10
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,objective,'reg:squarederror'


In [13]:
print(f'Melhores parâmetros encontrados: {grid_search_xgboost.best_params_}')


Melhores parâmetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}


In [14]:
predicao_xgb = grid_search_xgboost.best_estimator_.predict(X_teste)
rmse = np.sqrt(mean_squared_error(y_teste, predicao_xgb))
r2 = r2_score(y_teste, predicao_xgb)

In [15]:
print("Métricas do melhor XGBoost no conjunto de teste:\n")
print(f'RMSE = {rmse}')
print(f'R2 = {r2}')

Métricas do melhor XGBoost no conjunto de teste:

RMSE = 0.18126215781291888
R2 = 0.595290083575637


#### Teste com Gridsearch no RandomForestRegressor

In [20]:
modelo_randomforest = RandomForestRegressor(random_state=0)
grid_parametros_forest = {'n_estimators': [50, 100, 200, 1000], 'max_depth': [None, 5, 10, 20]}
grid_search_forest= GridSearchCV(modelo_randomforest, grid_parametros_forest, cv = 10, scoring='r2', n_jobs=-1)
grid_search_forest.fit(X_treino,y_treino)

,estimator,RandomForestR...andom_state=0)
,param_grid,"{'max_depth': [None, 5, ...], 'n_estimators': [50, 100, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,10
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,1000


In [21]:
print(f'Melhores parâmetros encontrados: {grid_search_forest.best_params_}')

Melhores parâmetros encontrados: {'max_depth': 10, 'n_estimators': 1000}


In [22]:
predicao_forest = grid_search_forest.best_estimator_.predict(X_teste)
rmse = np.sqrt(mean_squared_error(y_teste, predicao_forest))
r2 = r2_score(y_teste, predicao_forest)

In [23]:
print("Métricas da melhor RandomForest no conjunto de teste:\n")
print(f'RMSE = {rmse}')
print(f'R2 = {r2}')

Métricas da melhor RandomForest no conjunto de teste:

RMSE = 0.18597365877471453
R2 = 0.5739776094051298


In [28]:
melhor_modelo = grid_search_xgboost.best_estimator_

# Testando o modelo com um exemplo: 

Adaptando o formato dos dados para as transformações feitas, de maneira modularizada

In [24]:
dado = {'Series_Title': ['The Shawshank Redemption'],
 'Released_Year': ['1994'],
 'Certificate': ['A'],
 'Runtime': ['142 min'],
 'Genre': ['Drama'],
 'Overview': ['Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'],
 'Meta_score': [80.0],
 'Director': ['Frank Darabont'],
 'Star1': ['Tim Robbins'],
 'Star2': ['Morgan Freeman'],
 'Star3': ['Bob Gunton'],
 'Star4': ['William Sadler'],
 'No_of_Votes': [2343110],
 'Gross': ['28,341,469']}

dado = pd.DataFrame(dado)

In [25]:
def separar_runtime(string): 
    minutos, _  = string.split()
    minutos = float(minutos)

    return minutos

def converter_formatacao(string):
    if string is not np.nan:
        return string.replace(',', '')
    return string
    
mapeamento_classificacao = {
    'U': 'Livre',
    'G': 'Livre',
    'Passed': 'Livre',
    'Approved': 'Livre',
    'PG': 'Orientacao parental recomendada',
    'UA': 'Orientacao parental recomendada',
    'PG-13': 'Orientacao parental recomendada',
    'R': 'Adulto',
    'A': 'Adulto',
    'TV-MA': 'Adulto',
    '16': 'Adulto',
    'TV-14': 'Orientacao parental recomendada',
    'GP': 'Livre',
    'Unrated': 'Nao classificado',}
    
lista_generos = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']
lista_classificacao = ['Certificate_Orientacao parental recomendada', 'Certificate_Livre', 'Certificate_Nao classificado']

def transformar_dataframe(df): # função auxiliar para transformar os dados baseado no processo padronizado anteriormente

    df['Released_Year'] = df['Released_Year'].astype(int)
    df['Runtime'] = df['Runtime'].apply(separar_runtime)
    df['Gross'] = df['Gross'].apply(converter_formatacao).astype('Int64')
    df['Certificate'] = df["Certificate"].map(mapeamento_classificacao)
    df['Certificate'] = df["Certificate"].fillna("Nao classificado")
    

    generos_encoded = pd.DataFrame(columns = lista_generos, index = df.index).fillna(0)
    classificacao_encoded = pd.DataFrame(columns = lista_classificacao, index = df.index).fillna(False)

    for indice, linha in df.iterrows():
        for genero in linha['Genre'].split(', '): 
            if genero in lista_generos:
                generos_encoded.loc[indice, genero] = 1

        classif = f"Certificate_{linha['Certificate']}" 
        if classif in lista_classificacao:
            classificacao_encoded.loc[indice, classif] = True
    

    df = pd.concat([df.drop('Genre', axis=1), generos_encoded], axis=1)
    df = pd.concat([df.drop('Certificate', axis=1), classificacao_encoded], axis=1)
    
    df['Imputed_Gross'] = df['Gross']
    df['Imputed_Meta_score'] = df['Meta_score']

    df = df[features]

    return df

dado_tratado = transformar_dataframe(dado)

C:\Users\iuri_\AppData\Local\Temp\ipykernel_1900\1704177121.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  generos_encoded = pd.DataFrame(columns = lista_generos, index = df.index).fillna(0)
C:\Users\iuri_\AppData\Local\Temp\ipykernel_1900\1704177121.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  classificacao_encoded = pd.DataFrame(columns = lista_classificacao, index = df.index).fillna(False)


In [29]:
nota_predita = melhor_modelo.predict(dado_tratado)
nota_predita

array([8.798927], dtype=float32)

O exemplo dado foi predito com nota 8.8 no IMDB, o que é um resultado razoável, visto que a nota real do filme é 9.3, segundo o link: 
https://www.imdb.com/pt/title/tt0111161

#### Exportando para .pkl

In [32]:
joblib.dump(melhor_modelo, 'melhor_modelo_xgboost.pkl')

['melhor_modelo_xgboost.pkl']